## 1、GPU利用率

### 1.1、常见的造成GPU利用率低的原因


本质是**CPU的计算**或**I/O**的环节耗时长，导致GPU利用率上不去

### 1.2、数据加载与处理的耗时

Dataloader的几个相关参数

- num_workers：线程数
- prefetch_factor：每个worker提前加载样本数，设置不合理可能会让CPU处理时GPU空闲
- pin_memory：直接将数据映射到 GPU 的相关内存块上，省掉一点数据传输时间

另外，数据处理函数逻辑太复杂也影响资源利用率，建议减少for/while循环。

### 1.3、减少I/O操作的耗时

- 模型保存不宜太频繁(--save_steps参数)
- 日志打印、指标上报、进度上报等不宜太频繁
- 存储介质对时延影响也很明显
    - 本地存储介质性能：SSD > ceph > cfs-1.5 > hdfs > mdfs
    - 网络存储：数据与计算最好在同城；排查路由、网络带宽等其它因素。
- 数据不宜分成太多小文件，会影响I/O性能（主要是图像处理场景）
- 分布式训练时要使用 DistributedDataParallel （Pytorch）
- 多机训练要启用 GDRDMA （英伟达的远程直接显存访问机制）

### 1.4 其它CPU计算耗时

主要是loss计算和metric计算的复杂度（常见的问题不涉及此处，主要是自定义loss或metric需要注意这个问题）

## 2、Batch的拼接方式（Padding在哪边）

<img src="training-batch.png" width=600px/>

<img src="inference-batch.png" width=600px/>

## 3、Tokenizer怎么分词，怎么处理未登录词

- 参考SentencePiece：https://github.com/google/sentencepiece
    - 双字节对儿（BPE）编码：高频二元组合并，低频的分开，迭代至不能在合为止，ChatGPT用的就是这个方法
    - 基于无监督学习的Unigram模型切分（在训练集上以极大似然概率训练一个分词模型）

## 4、数据准备与处理

### 4.1、数据采集

- 自然来源（如业务日志）：真实数据
- Web抓取：近似数据
- 人造

### 4.2、数据标注

- 专业标注公司
   - 定标准，定验收指标
   - 预标注
   - 反馈与优化
   - 正式标注
   - 抽样检查：合格->验收；不合格->返工
- 众包
   - 定标准，定检验指标
   - 抽样每个工作者的质量
   - 维系高质量标注者社区
- 主动学习：通过模型选择重要样本，由专家标注，再训练模型
- 设计产品形态，在用户自然交互中产生标注数据（例如点赞、收藏）

### 4.3、数据清洗

- 去除不相关数据
- 去除冗余数据（例如重复的样本）
- 去除误导性数据（业务相关）

### 4.4、样本均衡性

- 尽量保证每个标签（场景/子问题）都有足够多的训练样本
- 每个标签对应的数据量尽量相当
   - 或者在保证每个标签样本充值的前提下，数据分布尽量接近真实业务场景的数据分布
- 数据不均衡时的策略
   - 数据增强：为数据不够类别造数据：（1）人工造；（2）通过模板生成再人工标注；（3）由模型自动生成（再人工标注/筛选）
   - 数据少的类别数据绝对数量也充足时，Downsample一般比Upsample效果好
   - 实在没办法的话，在训练loss里加权（一般不是最有效的办法）
- 根据业务属性，保证其他关键要素的数据覆盖，例如：时间因素、地域因素、用户年龄段等
 
### 4.5、数据集构建

- 数据充分的情况下
   - 切分训练集（训练模型）、验证集（验证超参）、测试集（检验最终模型+超参的效果）
   - 以随机采样的方式保证三个集合的数据分布一致性
   - 在以上三个集合里都尽量保证各个类别/场景的数据覆盖
- 数据实在太少
   - 交叉验证